# Data Cleaning

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

os.chdir('/content/drive/MyDrive/IronInsight')
os.getcwd()

'/content/drive/MyDrive/IronInsight'

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('data/openpowerlifting-2024-09-21.csv')

<ipython-input-4-146bac32c525>:1: DtypeWarning: Columns (31,33,35,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/openpowerlifting-2024-09-21.csv')


In [ ]:
pd.set_option('display.max_columns', 6)
df

,Name,Sex,Event,...,MeetTown,MeetName,Sanctioned
0,E.S. Denisenko,F,B,...,Luninets,Bison Power Cup,Yes
1,I.S. Lebetskaya,F,B,...,Luninets,Bison Power Cup,Yes
2,K. Yakimovich,F,B,...,Luninets,Bison Power Cup,Yes
3,A.G. Golneva,F,B,...,Luninets,Bison Power Cup,Yes
4,E.V. Marunevskaya,F,B,...,Luninets,Bison Power Cup,Yes
...,...,...,...,...,...,...,...
3290030,Ibrahim Sandogji,M,SBD,...,Riyadh,The Kingdom Classic Powerlifting Championships,Yes
3290031,Feras Alwahhabi,M,SBD,...,Riyadh,The Kingdom Classic Powerlifting Championships,Yes
3290032,Musab Alessa,M,SBD,...,Riyadh,The Kingdom Classic Powerlifting Championships,Yes
3290033,Ali Alholan,M,SBD,...,Riyadh,The Kingdom Classic Powerlifting Championships,Yes


# Raw Data Analysis

In [ ]:
print(df.shape) # 3,290,035 enteries in raw data, 42 features
df.columns

(3290035, 42)


Index(['Name', 'Sex', 'Event', 'Equipment', 'Age', 'AgeClass',
       'BirthYearClass', 'Division', 'BodyweightKg', 'WeightClassKg',
       'Squat1Kg', 'Squat2Kg', 'Squat3Kg', 'Squat4Kg', 'Best3SquatKg',
       'Bench1Kg', 'Bench2Kg', 'Bench3Kg', 'Bench4Kg', 'Best3BenchKg',
       'Deadlift1Kg', 'Deadlift2Kg', 'Deadlift3Kg', 'Deadlift4Kg',
       'Best3DeadliftKg', 'TotalKg', 'Place', 'Dots', 'Wilks', 'Glossbrenner',
       'Goodlift', 'Tested', 'Country', 'State', 'Federation',
       'ParentFederation', 'Date', 'MeetCountry', 'MeetState', 'MeetTown',
       'MeetName', 'Sanctioned'],
      dtype='object')

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])

# Find the earliest date
earliest_date = df['Date'].min()
latest_dat = df['Date'].max()
print(earliest_date)
print(latest_dat)

1964-09-05 00:00:00
2024-09-15 00:00:00


In [ ]:
print(df['Country'].unique())
len(df['Country'].unique()) # 195 countries and territories

[nan 'Belarus' 'USA' 'Finland' 'Canada' 'Czechia' 'Estonia' 'Hungary'
 'Brazil' 'Russia' 'Germany' 'Sweden' 'Iceland' 'Spain' 'Poland'
 'Kazakhstan' 'Portugal' 'Croatia' 'UK' 'Norway' 'Japan' 'Ireland'
 'Belgium' 'Austria' 'France' 'Australia' 'Latvia' 'Lithuania'
 'Saudi Arabia' 'Jordan' 'UAE' 'Iraq' 'Syria' 'Venezuela' 'Colombia'
 'Cuba' 'Chile' 'Honduras' 'Mexico' 'Guatemala' 'El Salvador' 'Argentina'
 'Nicaragua' 'Costa Rica' 'Ecuador' 'Haiti' 'Iran' 'Malaysia' 'Indonesia'
 'Uzbekistan' 'Vietnam' 'Bahrain' 'China' 'India' 'Azerbaijan' 'Algeria'
 'Ukraine' 'Turkey' 'Thailand' 'Peru' 'Cameroon' 'Egypt' 'South Korea'
 'Italy' 'Philippines' 'Ivory Coast' 'South Africa' 'Nigeria' 'Greece'
 'Libya' 'Dominican Republic' 'Georgia' 'Armenia' 'Panama' 'Turkmenistan'
 'Kenya' 'Ghana' 'Scotland' 'Serbia' 'Moldova' 'Mongolia' 'New Zealand'
 'Singapore' 'Kyrgyzstan' 'The Gambia' 'England' 'Morocco' 'Taiwan'
 'Cyprus' 'Netherlands' 'Israel' 'Papua New Guinea' 'Namibia' 'Hong Kong'
 'Tajikistan' '

195

# Clean Data

* Remove any invalid benches
* Remove any non-tested meets
* Keep only benches with "Raw" and "Wraps" for Equipment
* Keep people who've competed multiple times
* Remove bench outliers and age outliers



In [ ]:
validBench = df.dropna(subset=['Best3BenchKg'])
validBench.shape # 2,907,058 enteries with valid bench

(2907058, 42)

In [ ]:
validBenchTested = validBench[validBench['Tested'] == 'Yes']
validBenchTested.shape # 2,195,945 enteries with valid bench and tested

(2195945, 42)

In [ ]:
validBenchTestedLimitEquip = validBenchTested[
    (validBenchTested['Equipment'] == 'Raw') |
    (validBenchTested['Equipment'] == 'Wraps')
]
validBenchTestedLimitEquip.shape #  1,042,599 enteries with valid bench, tested, and only Raw and Wraps for Equipment

(1042599, 42)

In [ ]:
columns_to_keep = [
    'Name', 'Sex', 'Age', 'BodyweightKg', 'WeightClassKg', 'Best3BenchKg', 'Date'
]
filteredValid = validBenchTestedLimitEquip[columns_to_keep]
filteredValid.head()

,Name,Sex,Age,BodyweightKg,WeightClassKg,Best3BenchKg,Date
666,Maysar Olaymi,M,24.0,43.5,49,91.0,2017-02-21
667,Mohammad Alshnaiti,M,15.0,39.5,49,70.0,2017-02-21
670,Ali Al-Darraji,M,35.0,52.3,54,142.0,2017-02-21
671,Osama Salah,M,21.0,56.5,59,85.0,2017-02-21
672,Ziad Androun,M,31.0,63.5,65,131.0,2017-02-21


In [ ]:
name_counts = filteredValid['Name'].value_counts()
multiple_names = name_counts[name_counts > 1].index
multipleNames = filteredValid[filteredValid['Name'].isin(multiple_names)]
multipleNames.shape

(862454, 7)

In [ ]:
count_male = (multipleNames["Sex"] == "M").sum()
count_female = (multipleNames["Sex"] == "F").sum()
count_other = (multipleNames["Sex"] == "Mx").sum()

print(count_male)
print(count_female)
print(count_other)

599652
262768
34


In [ ]:
multipleNames = multipleNames[multipleNames["Best3BenchKg"] >= 0] # remove bench negatives
multipleNames.rename(columns={'Best3BenchKg': 'BestBench'}, inplace=True)
print(multipleNames["BestBench"].max())
print(multipleNames["BestBench"].min())
multipleNames.shape

320.0
1.0


<ipython-input-15-2e9b32219daa>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  multipleNames.rename(columns={'Best3BenchKg': 'BestBench'}, inplace=True)


(862152, 7)

In [ ]:
multipleNames = multipleNames[multipleNames["Age"] <= 110] # outliers
multipleNames = multipleNames[multipleNames["Age"] >= 14]
age_counts = multipleNames["Age"].value_counts()
print(age_counts.max())
print(age_counts.min())
print(age_counts)
multipleNames.shape

24670
1
Age
22.5    24670
21.5    24402
20.5    23297
17.5    21544
19.5    20606
        ...  
90.5        3
93.5        3
92.0        2
94.5        1
94.0        1
Name: count, Length: 162, dtype: int64


(729212, 7)

In [ ]:
bodyweights = multipleNames["BodyweightKg"]
print(bodyweights.max())
print(bodyweights.min())

260.2
25.63


In [ ]:
# some instances where weight class ends in + (i.e. 107+)
multipleNames['WeightClassKg'] = (
    multipleNames['WeightClassKg']
    .str.rstrip('+')               # Remove trailing '+'
    .replace('', np.nan)           # Replace empty strings with NaN
)

# Convert the column to numeric (float)
multipleNames['WeightClassKg'] = pd.to_numeric(multipleNames['WeightClassKg'], errors='coerce')
multipleNames['WeightClassKg']

,WeightClassKg
666,49.0
667,49.0
670,54.0
672,65.0
673,65.0
...,...
3290029,120.0
3290030,66.0
3290031,83.0
3290032,93.0


In [ ]:
name_counts = multipleNames['Name'].value_counts()
multiple_names = name_counts[name_counts > 1].index
multipleNames = multipleNames[multipleNames['Name'].isin(multiple_names)]

name_counts = multipleNames["Name"].value_counts()
ave = name_counts.mean()

print("Average number of meets a single person competed in: ", ave)
print(name_counts)

Average number of meets a single person competed in:  4.497087498839333
Name
Gary Teeter              197
Magomedamin Israpilov    118
Matt Slate               110
Vladimir Chubarov        101
Bob Legg                 101
                        ... 
Rachel Hare                2
Madison Weigandt           2
Morgan Mayo                2
Tianna Richardson          2
Mostafa Ali Alfaky         2
Name: count, Length: 161545, dtype: int64


In [ ]:
cleanedData = multipleNames
cleanedData.reset_index(drop=True, inplace=True)
cleanedData

,Name,Sex,Age,BodyweightKg,WeightClassKg,BestBench,Date
0,Maysar Olaymi,M,24.0,43.5,49.0,91.0,2017-02-21
1,Mohammad Alshnaiti,M,15.0,39.5,49.0,70.0,2017-02-21
2,Ali Al-Darraji,M,35.0,52.3,54.0,142.0,2017-02-21
3,Ziad Androun,M,31.0,63.5,65.0,131.0,2017-02-21
4,Hasan Al-Tameemi,M,39.0,64.4,65.0,157.0,2017-02-21
...,...,...,...,...,...,...,...
726477,Mostafa Ali Alfaky,M,35.0,146.3,120.0,155.0,2022-10-13
726478,Ibrahim Sandogji,M,17.0,64.4,66.0,120.0,2022-10-13
726479,Feras Alwahhabi,M,17.0,82.9,83.0,112.5,2022-10-13
726480,Musab Alessa,M,17.0,91.3,93.0,130.0,2022-10-13


In [ ]:
mayser_olaymi = cleanedData[cleanedData['Name'] == 'Maysar Olaymi']

mayser_olaymi

,Name,Sex,Age,BodyweightKg,WeightClassKg,BestBench,Date
0,Maysar Olaymi,M,24.0,43.50,49.0,91.0,2017-02-21
1338,Maysar Olaymi,M,25.0,44.44,49.0,107.0,2018-02-17
1717,Maysar Olaymi,M,26.0,44.80,49.0,125.0,2019-04-25
2596,Maysar Olaymi,M,25.0,45.78,49.0,119.0,2018-10-07
3887,Maysar Olaymi,M,23.0,43.61,49.0,101.0,2017-02-25
7674,Maysar Olaymi,M,23.0,42.52,49.0,85.0,2016-02-15


In [ ]:
cleanedData['BodyweightKg'] = cleanedData['BodyweightKg'].fillna(cleanedData['WeightClassKg'])
cleanedData = cleanedData.dropna(subset=['BodyweightKg'])
cleanedData['BodyweightKg'] = pd.to_numeric(cleanedData['BodyweightKg'], errors='coerce')
cleanedData['Date'] = cleanedData['Date'].apply(lambda x: pd.to_datetime(x))
cleanedData

<ipython-input-22-7ee72f498325>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleanedData['BodyweightKg'] = pd.to_numeric(cleanedData['BodyweightKg'], errors='coerce')
<ipython-input-22-7ee72f498325>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleanedData['Date'] = cleanedData['Date'].apply(lambda x: pd.to_datetime(x))


,Name,Sex,Age,BodyweightKg,WeightClassKg,BestBench,Date
0,Maysar Olaymi,M,24.0,43.5,49.0,91.0,2017-02-21
1,Mohammad Alshnaiti,M,15.0,39.5,49.0,70.0,2017-02-21
2,Ali Al-Darraji,M,35.0,52.3,54.0,142.0,2017-02-21
3,Ziad Androun,M,31.0,63.5,65.0,131.0,2017-02-21
4,Hasan Al-Tameemi,M,39.0,64.4,65.0,157.0,2017-02-21
...,...,...,...,...,...,...,...
726477,Mostafa Ali Alfaky,M,35.0,146.3,120.0,155.0,2022-10-13
726478,Ibrahim Sandogji,M,17.0,64.4,66.0,120.0,2022-10-13
726479,Feras Alwahhabi,M,17.0,82.9,83.0,112.5,2022-10-13
726480,Musab Alessa,M,17.0,91.3,93.0,130.0,2022-10-13


In [ ]:
print(cleanedData['Age'].dtype)
print(cleanedData['BodyweightKg'].dtype)
print(cleanedData['WeightClassKg'].dtype)
print(cleanedData['BestBench'].dtype)
print(cleanedData['Date'].dtype)

float64
float64
float64
float64
datetime64[ns]


In [ ]:
#cleanedData.to_csv("cleaned_data_v3.csv", index=False)